<a href="https://colab.research.google.com/github/vifirsanova/hse-python-course/blob/main/compression/quantization_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 90.5 MB/s eta 0:00:00


In [2]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


NF4 (Normal-Float 4-bit):
- квантизация + распределение, восстанавливающее исходное распределение весов
- преимущество перед стандартной квантизацией: точнее, чем нормальное распределение
- доказано по критерию Шапиро-Уилка

In [17]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False
)

model_4bit = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/polyglot-ko-5.8b",
    device_map="auto",
    quantization_config=quantization_config
)

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/36.8k [00:00<?, ?B/s]

model-00001-of-00013.safetensors:   0%|          | 0.00/926M [00:00<?, ?B/s]

model-00002-of-00013.safetensors:   0%|          | 0.00/952M [00:00<?, ?B/s]

model-00003-of-00013.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

...

model-00011-of-00013.safetensors:   0%|          | 0.00/952M [00:00<?, ?B/s]

model-00012-of-00013.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00013-of-00013.safetensors:   0%|          | 0.00/515M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Здесь для загрузки очень большой модели используется *шардинг*, разделение весов модели на чанки для оптимизации процесса загрузки

In [22]:
sample_layer_weight = model_4bit.gpt_neox.layers[0].attention.query_key_value.weight

In [23]:
sample_layer_weight.dtype

torch.uint8

In [24]:
sample_layer_weight

Parameter containing:
Parameter(Params4bit([[103],
            [184],
            [110],
            ...,
            [  9],
            [ 58],
            [125]], device='cuda:0', dtype=torch.uint8))

In [25]:
sample_layer_weight.shape

torch.Size([25165824, 1])

- dtype == torch.uint8 => в одном тензоре хранится 2 4-битных значения
- 2 * 25 165 824 = 50 331 648 тензоров
- 50 331 648 = 3 * 4096 * 4096
- размерность исходной матрицы весов GPTNeoX `3 x hidden_dim x hidden_dim `
- т.е. после квантизации исходные веса обращены в одномерный тензор

In [27]:
[v[0] for v in sample_layer_weight[:2].cpu().numpy().tolist()]

[103, 184]

In [28]:
format(sample_layer_weight[0].item(), '08b'), format(sample_layer_weight[1].item(), '08b')

('01100111', '10111000')

- исходный тензор `01100111`
- представлен как пара `0110` и `0111`

Полная деквантизация: https://id2thomas.medium.com/ml-bitsandbytes-nf4-quantize-dequantize-analysis-1ad91d9912c9